# Anatomy of Redis Web Application

- login cookies
- shopping cart cookies
- caching generated web pages
- caching database rows
- analysing web page visits

Based on the tutorial from [here](https://redis.com/ebook/part-1-getting-started/chapter-2-anatomy-of-a-redis-web-application/).

## Login Cookies

We can use hash to store the mapping of login user token to the user. 

To set the token:

```
HSET login: token123 user1
ZADD recent: token123 1565576538
```

To get the token:

```
HGET login: token123
```


In [1]:
import time

import redis

In [2]:
conn = redis.Redis(decode_responses=True)

In [26]:
conn.ping()

True

In [27]:
# check_token checks if the token exists.
def check_token(conn, token):
    return conn.hget("login:", token)

In [28]:
def update_token(conn, token, user, item=None):
    timestamp = time.time()

    # Keep a mapping from the token to the logged-in user.
    conn.hset("login:", token, user)

    # Record when the token was last seen.
    conn.zadd("recent:", {token: timestamp})

    # Record that the user viewed the item.
    if item:
        conn.zadd("viewed:" + token, {item: timestamp})

        # Remove old items, keeping the most recent 25.
        conn.zremrangebyrank("viewed:" + token, 0, -26)

In [29]:
update_token(conn, "token123", "user123", "books")

In [30]:
check_token(conn, "token123")

'user123'

In [31]:
conn.zrange("recent:", 0, -1, withscores=True)

[('token123', 1672990183.4007082)]

In [34]:
conn.zrange("recent:", 1, -1, withscores=True, desc=True)

[]

In [4]:
QUIT = False
LIMIT = 1_000_000


def clean_session(conn):
    while not QUIT:
        size = conn.zcard("recent:")
        if size <= LIMIT:
            time.sleep(1)
            continue

        # Here, the keys already exceeded the LIMIT.
        # We want to delete the additional keys, but only max 1,000 keys at a time every second.
        # The scores are sorted in ascending order.
        end_index = min(size - LIMIT, 1_000)
        tokens = conn.zrange("recent:", 0, end_index - 1)

        session_keys = []
        for token in tokens:
            session_keys.append("viewed:" + token)

        conn.delete(*session_keys)
        conn.hdel("login:", *tokens)
        conn.zrem("recent:", *tokens)

In [ ]:
# Alternative to the implementation below is to sort the scores in descending.
# There can be a lot of tokens to delete, so we limit it to max 1,000 at a time.
STORE_LIMIT = 1_000_000
DELETE_LIMIT = 1_000

tokens = conn.zrange("recent:", STORE_LIMIT, STORE_LIMIT + DELETE_LIMIT, desc=True)
conn.delete(*tokens)